# Overview

## Properties of remote computing

Resources such as compute and storage can be accessed anywhere, virtually unlimited, data is permanent (resilient against crashes), easily shareable, many different types of compute resources

## Characteristics of Cloud based Computing

On-demand resources (pay as you go)   
Resource Scalability (increased resources allocated to you as you need more)  
API-based access

# Virtual Machines

Virtual Machine is a process which is an OS running on existing system's OS.  
The existing system's OS is referred to as the **hypervisor**. 
VMs are useful because:
1. Easy way to use different OS
2. Fully utilize node capacity
3. Isolation- run shitty code on separate VMs

Key aspects are Host (physical machine), guest (VM on some OS), VMM/Hypervisor (software layer performing virtualization of the host)

## Virtualization

Suite of techniques that enable multiple Guests each running a separate OS to be run on the same physical machine.   
Challenges: 
1. Isolate guests from each other
2. Support different OS
3. Keep performance overhead small  
  
Full Virtualization - exact copy of the host hardware exposed to guest OS - no change required to guest OS, hard for x86    
Paravirtualization - similar to full virtualization, but not the exect copy of the host hardware is exposed, requires modification to Guest OS but no modification to guest applications

## XEN

Hypervisor that uses paravirtualization technique, requires modification of guest OS.   
XEN runs as the most priveleged software (ring 0), whereas guest OS runs at a lower privelege level (ring 1)

### XEN and syscalls

Naive setup: Guest OS calls XEN to execute each exception, XEN calls processor to execute syscall, processor executes syscall and returns control to XEN which returns control to OS.  
**This is inefficient as fuck**  
Faster exception handlers: Guest OS calls XEN to register an exception handler table. XEN validates the table, and places it in memory where the processor can find it. The guest OS executes a system call, and the processor directly accesses the system call handler table without XEN, and returns back to XEN after completion.   
**The idea here is we removed a level of abstraction, XEN, by storing exception handler table into memory, which made syscalls/exceptions much faster**

### XEN and Page Faults

What is a page fault? When the CPU tries to access a page not mapped in the virtual address space of a process 
Normal demand paging:  
Page table used to translate virtual addresses to physical addresses. Each process needs its own table, and the OS setsup the page table for a process when it starts the process.

### Normal Page fault handling

1. MMU notices that a particular virtual address cannot be translated to a physical address, and then notifies the CPU about this.
2. CPU sends an interrupt to the OS called the page fault
3. OS page fault handler is at a fixed memory location, and control transferred to this page fault handler.  
4. OS uses a page replacement algorithm to determine whether or not a page will be replaced and written to disk or not.
5. It reads the new code into that newly emptied page frame, and updates the page table entry. It then re-executes the faulting instruction, which should work.  
  
CPU works with virtual addresses, MMU contains page table per process which is used to translate virtual addresses to physical addresses, and this page table is set up by the OS. When a page fault occurs, the OS helps resolve it.

### XEN page fault handling

1. Guest OS registers page table with XEN
2. Processor encounters page fault, and transfers control to XEN (hardware exception table set to do this at boot time)
3. XEN reads the faulting page address from the processor register and saves it in the execution stack frame of the guest OS, and resolves it
4. XEN restarts the faulting instruction from the execution stack frame of the guest OS   
  
CPU works with virtual addresses, Guest OS registers per process page table with XEN, XEN sets up the page table inside MMU when it schedules a guest OS. MMU contains page table for a process executing within the Guest OS. At boot time, XEN sets the page fault handler to transfer control to itself. WHen the CPU encounters a page fault, control is transferred to XEN and XEN resolves the page fault

### Virtualizing I/O

XEN manages I/O devices, and the guest OS calls XEN for I/O.   
XEN uses shared memory regions between itself and the guest OS as a fast data transfer mechanism, and there's an asynchronous event mechanism much like DMA to notify the guest OS when I/O is complete

### Terms and Concepts

Domains: Guest OS/VMs are called Domains.  
Domain0: priveleged comain created by XEN on startup, control center for creating and managing other domains

### VMs in the Cloud

Must support different hardwars and supervisors, and support the provisioning of Virtual Machines remotely (i.e. allow API access to spin up VMs)

### Libvirt

Node/Host: Physical Machine   
Hypervisor: Layer of software that virtualizes a node in a set of virtual machines    
Domain: An instance of an OS running on a Node   
Goal: provide a common and stable layer sufficient to securely manage domains on a node.   
  
Libvirt daemon (background process - remember you can run things in the background with &)   
All interactions go through this daemon, and local communication happens over Unix sockets. Many available methods, and various Domains (Guest OS) defined using XML

# EC2

Instance Type- determines the hardware of the host computer used for launching your instances.  
Instances- Running VM.    
AMI- OS definition, multiple instatnces can be launched from the same AMI  
Goal of instance types is to support different kinds of applications.   
Perks: General purpose, compute and memory optimized, accelerated computing, storage optimized

### AMI - Amazon Machine Image

OS image that should be used for booting VM.  
Information included is hardware architecture on which AMI is built, virtualization type for which the AMI is built, where to find root volume, which contains instructions on how to boot the AMI

### EC2 run_instances

1. Find out a hardware node corresponding to the provided instance type and has enough capacity
2. Fetch specified AMI
3. Create volumes on the Elastic Block Storage corresponding to the block device mapping
4. Invoke the libvirt driver on the selected Node to start a domain using the AMI fetched

### EC2 Virtualization

Paravirtual, and HVM (Hardware virtual machine)- uses special virtualization extensions, allows running guest OS without any modifications

### EC2 Instance initialization

Initializing an instance at startup contains user data and is done via cloud-init. User data is used when initialization is done using bash scripts. Logs stored in /var/log/cloud-init-output.log.   
Various use cases including an instance with a specific applicatino running, enabling remote logins, injecting app specific config files

### Security Groups

Security groups act as a virtual firewall, you can choose to allow traffic on specific ports. When you launch an instqance, you can associate one or more security groups with an instance. Rules need to hbe added to the security groups that control traffic to/from the instance.   
By default, all outbound traffic is allowed, you can allow certain protocols, port ranges, source/destinations, and all traffic (0.0.0.0/0)   
Each account gets a default security group per VPC (Virtual private cloud, abstraction for managing set of AWS resources as a unit. During instance creation is is inside the default VPC).

### CIDR

IP Address: 32 bit, divided into Network number and host number.   
CIDR Addressing: 
IP and Mask, mask is used to obtain network number. Indicates how many most significant bits should be treated as 1 while filtering.  
Written as IP Address/Mask: 1272.16.0.0/16: Mask is 16 bits in size, 16 most significant bits should be treated as 1.   
Mask is used in routing table to determine how to route a packet, used in security group rules to detemine wheter to accept a packet

### SSH Key Pair

Use an SSH key pair to connect to the EC2 instance, Public key stored by Amazon, private key stored by you.  
When the user logs in:
1. EC2 instance is authenticated by verifying the host key
2. User SSH client creates a unique message and encrypts it with the User's private key
3. EC2 instance uses User's public key to verify the message by decrypting it

# IAM

## Authentication

System generates identity credentials for you, you present them to the system whenever required

## Authorization

What are you allowed to do?  
System defines permissions corresponding to your identity. System encodes permissions in Authorization Policies, when you invoke any action, the system verifies that your identity is allowed to perform the action according to Authorization Policies.   
Authorization checking comes after authentication is successful

## Unix Identities

Users: individuals, processes  
Group: mechanism to define permissions for a set of users. Effective permissions that a user gets is union of the user's individual permissions and all the permissions for the grups that the user is part of. Users are added to a group and they remain in that group forever.

## DB Systems / Workflow Systems

Access control decisions based on the functions a user is allowed to perform within an organization.   
Mechanism to introduce dynamism into User's permissions. Users are added/removed from a Role by the sysadmin. When a user is in a role, the effective permissions of the User are permissions available in that Role

## AWS Authentication and Authorization

Authentication done using:
1. Username and password for the web console
2. Access key for programmatic access, stored in ~/.aws/credentials

## Request Signing

Verifying that the HTTP request came from authenticated user, contains a MAC (Message Authentication code), shared secret key with the Access Key and ID

## IAM Identities

AWS Account root user - Email address and password used to sign up with AWS, use root user to cerate other IAM Users.  
IAM Users- represents person or service that interacts with AWS (name and password to sign into AWS Console, Access Keys for programmatic access)   
IAM Groups- collection of IAM users.   
Roles- permissions for IAM users

IAM Users are part of an AWS account, contain a resource URN (called ARN) in the following format: arn:aws:iam::<account-ID-without-hyphens>:user/<username>.  
Credentials contain access keys, passswords and username for console. Default permissions is nothing, but can be individually assigned by root user

Service Account - IAM user that is created solely for programmatic access is called a service account, two keys allowed at a time, and key rotation exists for creating new keys

IAM Group is a collection of users that allows you to specify permissions for multiple users, making it easier to manager permissions for those users. A group can contain many users and a user can belong to multiple groups. Groups can't be nested, can only contain users.

## Permissions and Policy

Permission is a action resouce pair. A policy is a dictates a when/who/where/how a permission is to be given out.  
Principal based Policies- policies that are attached to a user     
Resource based policies- policies attached to a resource

## Policy Elements

1. Effect: Allow/Deny
2. Action: Service Name: Action Name pair. Case insensitive, and each AWS service specifies the actions it supports. Multiple actions is supported, and you can use * to represent all actions of a service.
3. Resource: Specifies the object of the polocy, specified using Amazon Resource Names, use * to indicate any resource.

## Policy Variables

You can use policy variables to reuse policy definitions rather than having to create 1000 of the same policy.  
Define a policy with policy variables and attach it to the user.   
When a request comes in, AWS will check what policy variables are defined in the policy and pass them to the policy enforcement engine, which will check values against the policy

## Use Cases

User logging into the AWS console, SDK/CLI into AWS, service to service authorization, mobile app to AWS service, Cross-account services, access control issues

## IAM Role

Set of permissions with an action resource pair.   
Who can assume a role?
1. IAM user with the same AWS account as the role
2. IAM user with the different AWS account as the role
3. Web service offered by AWS
4. External user authenticated using external identity provider.  
  
Role does not have password or access keys, when a user assumes a role, temporary credentials are created and provided to the user   
Contains a permission policy (has a set of permissions for Role), and a trust policy (who can assume the role), permissions to pass the role to someone else.   


## IAM Roles and EC2

Special role for EC2 instance, given to each instance. Role defines authorization policies for any application running on the EC2 instance. Application can obtain temporary credentials by querying instance metadata service from instance.  
Instance metadata corresponding to a launched instance, accessible inside running instance i.e. curl http://169.254.169.254/latest/meta-data/

## Web identity Federation

Use it to authenticate application using identity provider, get a security token after authentication is done, use token to make calls to S3, typically uses OAuth 2.0